In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1)

In [4]:
X,y=mnist['data'],mnist['target']

### PCA General Algoritham

In [6]:
X_centered = X - X.mean(axis=0) # Center it around it mean

In [10]:
U,s,Vt = np.linalg.svd(X_centered)

MemoryError: 

In [11]:
w2 = Vt.T[:,:2]
X2D = X_centered.dot(w2)

NameError: name 'Vt' is not defined

# PCA

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [13]:
X2D

array([[ 122.26049662, -316.25784546],
       [1010.4939901 , -289.96085747],
       [ -58.99460025,  393.69456007],
       ...,
       [-271.50841746,  590.08259671],
       [-310.22778458, -116.71719422],
       [1058.86220328,  -83.39470127]])

In [15]:
pca.components_.T[:,0] # First Principal Component

array([-6.69767802e-18, -1.19014212e-18,  3.96576004e-19, -1.08053634e-18,
       -1.76774072e-20, -1.28354405e-20, -1.52911375e-20,  1.09602580e-20,
        1.98312740e-21,  2.55136923e-21,  1.96744801e-22,  0.00000000e+00,
       -9.45586728e-07, -3.72375478e-06, -1.83974638e-06, -7.66560990e-08,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.16825224e-07,  4.29493257e-07,  4.87706873e-06,  1.65230025e-05,
        2.20363273e-05,  3.88808632e-05,  7.40608353e-05,  8.83249624e-05,
        8.08498264e-05,  8.95696326e-05,  1.06176894e-04,  7.93600758e-05,
        3.60814384e-05,  3.08093056e-05,  2.39078689e-05,  3.64545496e-06,
       -2.94189031e-06,  1.88453120e-06,  1.14902709e-06,  4.83417621e-07,
        0.00000000e+00,  

In [16]:
pca.explained_variance_ratio_

array([0.09746116, 0.07155445])

### Right Number of Dimensions

In [17]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95)+1
d

154

In [19]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)
X_reduced.shape

(70000, 154)

### For Compression

In [20]:
pca = PCA(n_components=154)
X_reduced = pca.fit_transform(X)
X_recovered = pca.inverse_transform(X_reduced)

In [23]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(X,X_recovered)
np.sqrt(mse)

14.79968282156946

### Randomized PCA

In [24]:
rnd_pca = PCA(n_components=154,svd_solver='randomized')
X_reduced = rnd_pca.fit_transform(X)

### Incremental PCA

In [25]:
from sklearn.decomposition import IncrementalPCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X,n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X)

### Kernal PCA

In [28]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components=2,kernel='rbf',gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

MemoryError: 

### Hyperparameter Tuning

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [30]:
clf = Pipeline([
    ('kpca',KernelPCA(n_components=2)),
    ('log_reg',LogisticRegression())
])

In [31]:
param_grid = [
    {
    'kpca__gamma': np.linspace(0.03,0.05,10),
    'kpca__kernel': ['rbf','sigmoid']
    }
]

In [32]:
grid_search = GridSearchCV(clf,param_grid,cv=3)
grid_search.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


MemoryError: 

In [33]:
grid_search.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

#### Kernel PCA Inverse Transform

In [34]:
rbf_pca = KernelPCA(n_components=2,kernel='rbf',gamma=0.0433,fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_pre_image = rbf_pca.inverse_transform(X_reduced)

MemoryError: 

In [35]:
mean_squared_error(X,X_pre_image)

NameError: name 'X_pre_image' is not defined

# LLE - Locally Linear Embedding

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=2,n_neighbors=10)
X_reduced = lle.fit_transform(X)